In [1]:
import numpy as np
np.random.seed(10)

In [2]:
import tensorflow as tf

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [3]:
from PIL import Image

In [ ]:
#抓取圖片
import catch_photo as cp
def dogs_photo(local_path,dog):
    cp.got_photo(local_path,dog)


In [4]:
import glob
#儲存圖片陣列
files = []
#抓取c0,c1,c2 folder的圖片
for i in range(0,3):
    files.append(glob.glob ('C:\pythonwork\photo\c'+str(i)+'\*.jpg'))
    
len(files[0])

1000

In [5]:
#訓練陣列
x_train=[]
#真實陣列
y_train=[]
count = 0
for i in range(0,3):
    for myFile in files[i]:
        count = count + 1
        #處理圖片
        image =Image.open(myFile)
        #L = 灰階
        image=image.convert("L")
        #寬30，長30
        new_width  = 30
        new_height = 30
        image = image.resize((new_width, new_height), Image.ANTIALIAS)
        
        x_train.append (image)
        y_train.append (i)

In [6]:
print(len(x_train))
print(len(y_train))
x_train[0]

2827
2827


In [7]:
#圖片轉四維
print(x_train[0])
a=np.array(x_train[0])
print(a.shape)
print(a.ndim)
n=[]
n.append(a)
print(len(n))
data=np.array(n)
data.shape

<PIL.Image.Image image mode=L size=30x30 at 0x259D41A47F0>
(30, 30)
2
1


(1, 30, 30)

In [8]:
#圖片轉4維
nd3d_list = []
for i in range(0, len(x_train)):
    nd3d_list.append(np.array(x_train[i]))
x_data = np.array(nd3d_list)
#(圖片數,寬,高,色彩維度)
x_data=x_data.reshape(len(x_data),30,30,1)

print("shape of x_data : ", x_data.shape)

shape of x_data :  (2827, 30, 30, 1)


In [9]:
nd3d_list_1 = []
for i in range(0, len(y_train)):
    nd3d_list_1.append(np.array(y_train[i]))
y_data = np.array(nd3d_list_1)

print("shape of y_data : ", y_data.shape)
y_data[0]
y_data.shape

shape of y_data :  (2827,)


(2827,)

In [11]:
from keras.utils import np_utils
#分三類
num_classes=3
#標準化
x_data=x_data.astype('float32')/255
#載入真實數據
y_data=np_utils.to_categorical(y_data,num_classes)

In [12]:
#使用keras模組
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D

In [13]:
#線性堆疊模組
model=Sequential()
#卷積層1 設定激活函數
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(30,30,1),activation='relu',padding='same'))
#放棄25%神經元以免overfitting
model.add(Dropout(0.25))

In [14]:
#池化層1
model.add(MaxPooling2D(2,2))

In [15]:
#卷積層2
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Dropout(0.25))

In [16]:
#池化層2
model.add(MaxPooling2D(2,2))

In [17]:
#平坦層
model.add(Flatten())
model.add(Dropout(0.25))

In [18]:
#隱藏層
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))

In [19]:
#輸出層
model.add(Dense(num_classes,activation='softmax'))

In [20]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        320       
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

In [21]:
#設定訓練
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
#開始訓練
#訓練資料占八成，週期為10，批次為2
train_history=model.fit(x_data,y_data,validation_split=0.2,epochs=10,batch_size=128,verbose=2)

W1230 15:26:45.429426 65868 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2261 samples, validate on 566 samples
Epoch 1/10
2261/2261 - 8s - loss: 1.0507 - accuracy: 0.5170 - val_loss: 1.9448 - val_accuracy: 0.0000e+00
Epoch 2/10
2261/2261 - 5s - loss: 0.8079 - accuracy: 0.6829 - val_loss: 1.6878 - val_accuracy: 0.0177
Epoch 3/10
2261/2261 - 5s - loss: 0.7185 - accuracy: 0.7218 - val_loss: 1.5461 - val_accuracy: 0.0972
Epoch 4/10
2261/2261 - 5s - loss: 0.6709 - accuracy: 0.7439 - val_loss: 1.1499 - val_accuracy: 0.4187
Epoch 5/10
2261/2261 - 5s - loss: 0.6522 - accuracy: 0.7621 - val_loss: 1.4794 - val_accuracy: 0.3004
Epoch 6/10
2261/2261 - 5s - loss: 0.6265 - accuracy: 0.7567 - val_loss: 1.5993 - val_accuracy: 0.2527
Epoch 7/10
2261/2261 - 5s - loss: 0.6111 - accuracy: 0.7665 - val_loss: 1.5166 - val_accuracy: 0.3127
Epoch 8/10
2261/2261 - 5s - loss: 0.5902 - accuracy: 0.7762 - val_loss: 1.2122 - val_accuracy: 0.4541
Epoch 9/10
2261/2261 - 5s - loss: 0.5985 - accuracy: 0.7687 - val_loss: 1.6070 - val_accuracy: 0.3180
Epoch 10/10
2261/2261 - 5s - lo

In [23]:
#儲存模型
model.save("dogs_model_1000.h5")
print("saved model to disk")

saved model to disk


In [24]:
label_dict={0:"黃金獵犬",1:"德國",2:"哈士奇"}

In [25]:
from PIL import Image
img = Image.open('img/2.JPEG')# image extension *.png,*.jpg
img=img.convert("L")
new_width  = 30
new_height = 30
img = img.resize((new_width, new_height), Image.ANTIALIAS)
img

In [26]:
#將圖片轉陣列
import numpy as np
sample=[]
sample.append(img)
a=np.array(sample[0])
print(a.shape)
a=a.reshape(1,30,30,1).astype("float32")/255
a.shape

(30, 30)


(1, 30, 30, 1)

In [27]:
import tensorflow.keras as keras
#載入模型
model = keras.models.load_model("dogs_model2.h5")

In [28]:
#a = 預測圖片
prediction=model.predict_classes(a)

In [29]:
print(label_dict[prediction[0]])
img

哈士奇


In [ ]:
from kivy.app import App
from kivy.lang import Builder
import os
from kivy.uix.button import Button
from kivy.uix.screenmanager import ScreenManager , Screen , FadeTransition
from kivy.uix.image import Image
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.gridlayout import GridLayout
from kivy.uix.pagelayout import PageLayout
from kivy.properties import ObjectProperty,StringProperty , ListProperty
from kivy.uix.widget import Widget
from glob import glob
from kivy.uix.popup import Popup
from kivy.uix.camera import Camera
from kivy.uix.progressbar import ProgressBar
from kivy.graphics import Color, Ellipse, Rectangle
import time
from kivy.clock import Clock
from kivy.graphics.texture import Texture
import cv2
import tensorflow.keras as keras


Builder.load_string("""
#:import F kivy.factory.Factory
#:import hex kivy.utils.get_color_from_hex
<f>:
    name:'main'
    canvas.before:
        Rectangle:
            pos: self.pos
            size: self.size
            
    FloatLayout:
        size_hint:1 , .3
        pos_hint:{'top':1}
        canvas:
            Color:
                rgba:hex('#006666')
            Rectangle:
                size:self.size
                pos:self.pos
    
        Label:
            pos_hint:{'x':0 , 'top':1}
            size_hint:.2 , None
            font_size:'40dp'
            text:'main'
            
        HSeparator:
            pos_hint:{'top':0}
            
            
    GridLayout:
        cols:4
        spacing:10
        padding:10
        size_hint:.5 , .2
        pos_hint:{'center_x':.5 ,'top':.9}

        Button:
            on_press:root.manager.current = 'third'
            background_normal:'photo_app/if__camera_2560451.png'

        Button:
            on_press: root.manager.current = 'other'
            background_normal:'photo_app/if_list_1167996.png'

        Button:
            on_press:root.manager.current = 'set'
            background_normal:'photo_app/if_Artboard_9_3030695.png'
            
        Button:
            on_press:root.manager.current = 'main'
            background_normal:'photo_app/if_home-80px_510929_down.png'
            
    RelativeLayout:
        pos_hint:{'x':0 , 'top':.6}
        size_hint:1 , .7
        id:Main_image_list
    
            
            
<s>:
    name:'other'
    canvas.before:
        Rectangle:
            pos: self.pos
            size: self.size
            
    FloatLayout:
        size_hint:1 , .3
        pos_hint:{'top':1}
        canvas:
            Color:
                rgba:hex('#006666')
            Rectangle:
                size:self.size
                pos:self.pos
    
        Label:
            pos_hint:{'x':0 , 'top':1}
            size_hint:.2 , None
            font_size:'40dp'
            text:'Picture'
            
        HSeparator:
            pos_hint:{'top':0}
    
        
    GridLayout:
        cols:4
        spacing:10
        padding:10
        size_hint:.5 , .2
        pos_hint:{'center_x':.5 ,'top':.9}

        Button:
            on_press:root.manager.current = 'third'
            background_normal:'photo_app/if__camera_2560451.png'


        Button:
            on_press: root.manager.current = 'other'
            background_normal:'photo_app/if_list_1167996_down.png'
            
        Button:
            on_press:root.manager.current = 'set'
            background_normal:'photo_app/if_Artboard_9_3030695.png'
            
        Button:
            on_press:root.manager.current = 'main'
            background_normal:'photo_app/if_home-80px_510929.png'

    ScrollView:
        size_hint:None, None
        size:500, 320
        pos_hint:{'center_x': .5, 'center_y': .4}
        do_scroll_x:False
        id:pic_list
        
<picture>:
    size:100, 100
    size_hint:None, None
    background_normal:root.source
    on_press:F.mypop(img = root.source).open()            

                
<third>:
    canvas.before:
        Color:
            rgba: 0, 0, 0, 1
        Rectangle:
            pos: self.pos
            size: self.size
    
    FloatLayout:
        Button:
            background_normal:'photo_app/if_aiga_lefrrow_inv_134126.png'
            size_hint:None ,None
            pos_hint:{'x':0 , 'top':1}
            on_press:root.manager.current = 'other'
            
    BoxLayout:
        orientation:'vertical'
        size_hint:1 , .8
        id:camera_site
        
        
            
        Button:
            background_normal:'photo_app/if_icon-ios7-circle-outline_211717.png'
            size_hint:None , None
            size:100 , 100
            pos_hint:{'center_x':.5 , 'y':1}
            on_press:root.onclick()
            on_release:F.mypop(img = root.source).open()
            
<setting>:
    canvas.before:
        Rectangle:
            pos: self.pos
            size: self.size
            
    FloatLayout:
        size_hint:1 , .3
        pos_hint:{'top':1}
        canvas:
            Color:
                rgba:hex('#006666')
            Rectangle:
                size:self.size
                pos:self.pos
    
        Label:
            pos_hint:{'x':0 , 'top':1}
            size_hint:.2 , None
            font_size:'40dp'
            text:'setting'
            
        HSeparator:
            pos_hint:{'top':0}

    GridLayout:
        cols:4
        spacing:10
        padding:10
        size_hint:.5 , .2
        pos_hint:{'center_x':.5 ,'top':.9}

        Button:
            on_press:root.manager.current = 'third'
            background_normal:'photo_app/if__camera_2560451.png'

        Button:
            on_press: root.manager.current = 'other'
            background_normal:'photo_app/if_list_1167996.png'

        Button:
            on_press:root.manager.current = 'set'
            background_normal:'photo_app/if_Artboard_9_3030695_down.png'
            
        Button:
            on_press:root.manager.current = 'main'
            background_normal:'photo_app/if_home-80px_510929.png'
            
        
            
<Separator@Widget>:
    canvas:
        Color:
            rgba:hex('#003333')
        Rectangle:
            size:self.size
            pos:self.pos
            
<HSeparator@Separator>:
    size_hint_y: None
    height: dp(2)
            
            
<mypop>:
    id:mypop
    title:''
    separator_height: 0
    size_hint:.7 , .7
    BoxLayout:
        orientation: 'vertical'
        Image:
            source:root.img
            size_hint: .5, .5 
            pos_hint:{'center_x': 0.5, 'center_y': 0.5}
            canvas.before:
                Color:
                    rgba: 1,1,1,1
                BorderImage:
                    source: 'shadow32.png'
                    border: (36,36,36,36)
                    size:(self.width*1.1, self.height*1)
                    pos:self.pos
                    
        BoxLayout:
            id:ans
            size_hint:.5 , .1
            pos_hint:{'center_x': 0.5}
            
                    

        BoxLayout:
            id:iden_test
            size_hint:1 , .1
            orientation: 'horizontal'

            Button:
                size_hint:.5 , 1
                text:'identify'
                on_release:root.pre()

            Button:
                size_hint:.5 , 1
                text:'back'
                on_release:root.dismiss()

<Main_Image>:
    source:root.Image_name
    size_hint:.5 ,.5
    
<CircularProgressBar>:
    size_hint: (None, None)
    height: 20
    width: 20
    
                
""")

class CircularProgressBar(ProgressBar):

    def __init__(self, **kwargs):
        super(CircularProgressBar, self).__init__(**kwargs)
        self.thickness = 10
        # Redraw on innit
        self.draw()

    def draw(self):

        with self.canvas:

            # Empty canvas instructions
            self.canvas.clear()

            # Draw no-progress circle
            Color(0.26, 0.26, 0.26)
            Ellipse(pos=self.pos, size=self.size)

            # Draw progress circle, small hack if there is no progress (angle_end = 0 results in full progress)
            Color(1, 0, 0)
            Ellipse(pos=self.pos, size=self.size,
                    angle_end=(0.001 if self.value_normalized == 0 else self.value_normalized*360))

            # Draw the inner circle (colour should be equal to the background)
            Color(0, 0, 0)
            Ellipse(pos=(self.pos[0] + self.thickness / 2, self.pos[1] + self.thickness / 2),
                    size=(self.size[0] - self.thickness, self.size[1] - self.thickness))


    def set_value(self, value):
        # Update the progress bar value
        self.value = value
        # Draw all the elements
        self.draw()


class Picture(Button):
    source = StringProperty(None)

class Main_Image(Image):
    Image_name = StringProperty(None)
    
    
class mypop(Popup):
    img = StringProperty(None)
    
    def loading(self):
        print("circle")
        self.progress = CircularProgressBar()
        self.ids.ans.add_widget(self.progress)
        Clock.schedule_interval(self.animate, 0.1)
        
    def animate(self, dt):
        if self.progress.value < 80:
            self.progress.set_value(self.progress.value + 1)
        else:
            self.progress.set_value(0)
    
    def close(self):
        self.close()
        
    def pre(self):
        import numpy as np
        np.random.seed(10)
        import tensorflow as tf
        from PIL import Image
        import glob
        #import tensorflow.keras as keras
        #from glob import glob
        model=keras.models.load_model("dogs_model.h5")
        label_dict={0:"retriever",1:"german sheepdog",2:"husky"}
        
        from PIL import Image
        img=Image.open(self.img)#隨便一張測試的
        img=img.convert("L")#灰階 
        new_w=30
        new_h=30
        img=img.resize((new_w,new_h),Image.ANTIALIAS)
        img
        import numpy as np
        sample=[]
        sample.append(img)
        a=np.array(sample[0])
        a=a.reshape(1,30,30,1).astype('float32')/255
        a.shape
        try:
            prediction=model.predict_classes(a)
        except:
            print("Unexpected error:", sys.exc_info()[0])
        """if prediction[0]==0:
            self.ids.ans.add_widget(Button(text='retriever'))

        elif prediction[0]==1:
            self.ids.ans.add_widget(Button(text='maltese'))

        elif prediction[0]==2:
            self.ids.ans.add_widget(Button(text='husky'))

        else:
            self.ids.ans.add_widget(Button(text='german sheepdog'))
        print(label_dict[prediction[0]])"""
        if prediction[0]==0:
            self.ids.ans.add_widget(Button(text='retriever'))
        elif prediction[0]==1:
            self.ids.ans.add_widget(Button(text='german sheepdog'))
        elif prediction[0]==2:
            self.ids.ans.add_widget(Button(text='husky'))
        img
        """files = []
        for i in range(0,2):
            files.append(glob.glob ('C:\pythonwork\crop\item'+str(i)+'\*.jpg'))#"crop_kidney\c1\image01.tif"

        '''curdir = 'C:\\pythonwork\\img'
        for i in glob(curdir+'\\*'):
            files.append(i)'''
        x_train=[]
        y_train=[]
        count = 0
        for i in range(0,2):
            for myFile in files[i]:
                #print(count, myFile)
                count = count + 1
                image =Image.open(myFile)
                image=image.convert("L")
                new_width  = 30
                new_height = 30
                image = image.resize((new_width, new_height), Image.ANTIALIAS)
                #image.resize()
                x_train.append (image)
                y_train.append (i)
        x_train[0]
        #圖片轉四維
        a=np.array(x_train[0])
        n=[]
        n.append(a)
        data=np.array(n)
        data.shape
        nd3d_list = []
        for i in range(0, len(x_train)):
            nd3d_list.append(np.array(x_train[i]))
        x_data = np.array(nd3d_list)
        x_data=x_data.reshape(len(x_data),30,30,1)
        nd3d_list_1 = []
        for i in range(0, len(y_train)):
            nd3d_list_1.append(np.array(y_train[i]))
        y_data = np.array(nd3d_list_1)

        y_data[0]
        y_data.shape
        from keras.utils import np_utils
        num_classes=2
        x_data=x_data.astype('float32')/255
        y_data=np_utils.to_categorical(y_data,num_classes)
        from keras.models import Sequential
        from keras.layers import Dense,Dropout,Activation,Flatten
        from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
        model=Sequential()
        model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(30,30,1),activation='relu',padding='same'))
        model.add(Dropout(0.25))
        model.add(MaxPooling2D(2,2))
        model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
        model.add(Dropout(0.25))
        model.add(MaxPooling2D(2,2))
        model.add(Flatten())
        model.add(Dropout(0.25))
        model.add(Dense(1024,activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(num_classes,activation='softmax'))
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        try:
            model=keras.models.load_model("my_model.h5")
            print("載入模型成功，繼續訓練模型")
        except:
            print("失敗，重新訓練型")
        train_history=model.fit(x_data,y_data,validation_split=0.2,epochs=10,batch_size=2,verbose=2)
        model.save("my_model.h5")
        print("saved model to disk")
        label_dict={0:"pan",1:"hat"}
        from PIL import Image
        img = Image.open(self.img)# image extension *.png,*.jpg
        
        print(self.img)
        new_width  = 30
        new_height = 30
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img=img.convert("L")
        import numpy as np
        sample=[]
        sample.append(img)
        a=np.array(sample[0])
        a=a.reshape(1,30,30,1).astype("float32")/255
        a.shape
        prediction=model.predict_classes(a)
        print(label_dict[prediction[0]])
        if label_dict[prediction[0]] == 'pan':
            self.ids.ans.add_widget(Button(text='pan'))
        else:
            self.ids.ans.add_widget(Button(text='hat'))
        img"""
        
        
class KivyCamera(Image):
    def __init__(self, capture, fps, **kwargs):
        super(KivyCamera, self).__init__(**kwargs)
        self.capture = capture
        Clock.schedule_interval(self.update, 1.0 / fps)
        

    def update(self, dt):
        ret, frame = self.capture.read()
        self.now_frame = frame
        if ret:
            # convert it to texture
            buf1 = cv2.flip(frame, 0)
            buf = buf1.tostring()
            image_texture = Texture.create(
                size=(frame.shape[1], frame.shape[0]), colorfmt='bgr')
            image_texture.blit_buffer(buf, colorfmt='bgr', bufferfmt='ubyte')
            # display image from the texture
            self.texture = image_texture
    
class f(Screen):
    def __init__(self , **kwargs):
        super(f , self).__init__(**kwargs)
        pass
        """grid = GridLayout(cols=2 , rows=2)
        for i in range(0,3):
            image = Main_Image(Image_name="img/{}.JPEG".format(i))
            grid.add_widget(image)
        self.ids.Main_image_list.add_widget(grid)"""

class s(Screen):
        
    def __init__(self , **kwargs):
        super(s , self).__init__(**kwargs)
        layout = GridLayout(cols=4, padding=10, spacing=10,
                size_hint=(None, None), width=500)
        layout.bind(minimum_height=layout.setter('height'))
        curdir = 'C:\\pythonwork\\img'
        '''for filename in glob(curdir+'\\*'):
            pic = picture(source=filename)
            layout.add_widget(pic)'''
        for i in glob(curdir+'\\*'):
            picture = Picture(source = i)
            layout.add_widget(picture)
        self.ids.pic_list.add_widget(layout)
        
class third(Screen):
    source = StringProperty(None)
    def __init__(self, **kwargs):
        super(third, self).__init__(**kwargs)
        self.capture = cv2.VideoCapture(0)
        self.camera = KivyCamera(capture=self.capture, fps=30)
        self.ids.camera_site.add_widget(self.camera)
        
    def onclick(self):
        timestr = time.strftime("%Y%m%d_%H%M%S")
        cv2.imwrite("img/IMG_{}.png".format(timestr) , self.camera.now_frame)
        self.source = "C:\pythonwork\img/IMG_{}.png".format(timestr)
        print('work')
        
class setting(Screen):
    pass
        
class sm(ScreenManager):
    def __init__(self , **kwargs):
        super(sm , self).__init__(**kwargs)
        self.add_widget(f(name="main"))
        self.add_widget(s(name="other"))
        self.add_widget(third(name="third"))
        self.add_widget(setting(name="set"))
        self.current = 'main'
    
class callimg(App):
    def build(self):
        return sm()

if __name__ == '__main__':
    callimg().run()

[INFO   ] [Logger      ] Record log in C:\Users\user\.kivy\logs\kivy_19-12-29_0.txt
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [1]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
from PIL import Image
import glob
import tensorflow.keras as keras
#from glob import glob
model=keras.models.load_model("dogs_model.h5")
label_dict={0:"retriever",1:"maltese",2:"husky",3:"german sheepdog"}
from PIL import Image
img=Image.open('img/0.JPEG')#隨便一張測試的
region1 = (500,0,650,150)
crop_Image=img
crop_Image=img.crop(region1)
crop_Image=crop_Image.convert("L")#灰階
crop_Image
new_w=30
new_h=30
crop_Image = crop_Image.resize((new_w,new_h),Image.ANTIALIAS)
#img=img.resize((new_w,new_h),Image.ANTIALIAS)
crop_Image
import numpy as np
sample=[]
#sample.append(img)
sample.append(crop_Image)
a=np.array(sample[0])
a=a.reshape(1,30,30,1).astype('float32')/255
a.shape
prediction=model.predict_classes(a)
if prediction[0]==0:
    print('0')
elif prediction[0]==1:
    print('1')

elif prediction[0]==2:
    print('2')

else:
    print('3')
print(label_dict[prediction[0]])
crop_Image

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1
maltese


In [9]:
#爬蟲
import requests

import urllib.request

from bs4 import BeautifulSoup

import os

import time




url_0='https://www.google.com/search?q=%E9%BB%83%E9%87%91%E7%8D%B5%E7%8A%AC&sxsrf=ACYBGNTCC10xmu_fstwDrTWf59iLqzKCAw:1577105733445&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiTmY_Y6MvmAhUIy4sBHUPSCxEQ_AUoAXoECBEQAw&biw=1536&bih=722'
url_1= 'https://www.google.com/search?biw=1536&bih=722&tbm=isch&sxsrf=ACYBGNSkWIzjGbTI7VI1mrQmnZSmS0SIXQ%3A1577105799212&sa=1&ei=h7kAXtjMDITdmAX28aT4AQ&q=%E5%BE%B7%E5%9C%8B%E7%89%A7%E7%BE%8A%E7%8A%AC&oq=%E5%BE%B7%E5%9C%8B&gs_l=img.3.0.35i39j0l9.43438.47372..48325...5.0..0.71.642.14......0....1..gws-wiz-img.....10..35i362i39j0i131j0i24.kAWHEjMXFFo'
url_2='https://www.google.com/search?biw=1536&bih=722&tbm=isch&sxsrf=ACYBGNTtWUF-DenSeITJhoYPb3IdLJ2GvQ%3A1577105848388&sa=1&ei=uLkAXtOxF7TVmAXxiZ6ADg&q=%E5%93%88%E5%A3%AB%E5%A5%87&oq=%E5%93%88%E5%A3%AB%E5%A5%87&gs_l=img.3..35i39j0l9.18566.20055..20470...1.0..0.61.446.10......0....1..gws-wiz-img.....10..35i362i39j0i24j0i131j0i3.rD415w8MKG8&ved=0ahUKEwjT7faO6cvmAhW0KqYKHfGEB-AQ4dUDCAc&uact=5'
photolimit = 50

url = []
url.append(url_0)
url.append(url_1)
url.append(url_2)
headers = {'User-Agent': 'Mozilla/5.0'}
for i in range(0,3):
    response = requests.get(url[i],headers = headers) #使用header避免訪問受到限制

    soup = BeautifulSoup(response.content, 'html.parser')

    items = soup.find_all('img')

    folder_path ='photo/c'+str(i)+'/'

    if (os.path.exists(folder_path) == False): #判斷資料夾是否存在

        os.makedirs(folder_path) #Create folder



    for index , item in enumerate (items):

        if (item and index < photolimit ):

            html = requests.get(item.get('src')) # use 'get' to get photo link path , requests = send request

            img_name = folder_path + str(index + 1) + '.png'



            with open(img_name,'wb') as file: #以byte的形式將圖片數據寫入

                file.write(html.content)

                file.flush()

            file.close() #close file

            print('第 %d 張' % (index + 1))

            time.sleep(1)



    print('Done')

第 1 張
第 2 張
第 3 張
第 4 張
第 5 張
第 6 張
第 7 張
第 8 張
第 9 張
第 10 張
第 11 張
第 12 張
第 13 張
第 14 張
第 15 張
第 16 張
第 17 張
第 18 張
第 19 張
第 20 張
Done
第 1 張
第 2 張
第 3 張
第 4 張
第 5 張
第 6 張
第 7 張
第 8 張
第 9 張
第 10 張
第 11 張
第 12 張
第 13 張
第 14 張
第 15 張
第 16 張
第 17 張
第 18 張
第 19 張
第 20 張
Done
第 1 張
第 2 張
第 3 張
第 4 張
第 5 張
第 6 張
第 7 張
第 8 張
第 9 張
第 10 張
第 11 張
第 12 張
第 13 張
第 14 張
第 15 張
第 16 張
第 17 張
第 18 張
第 19 張
第 20 張
Done
